In [9]:
import sys
import os

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now use absolute import
from app.config import DATA_DIR, ROOT_DIR

In [10]:
%load_ext autoreload
%autoreload 2


import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np

# from ..app.config import DATA_DIR

parent_dir = os.path.abspath(os.path.join('..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import base64
from dotenv import load_dotenv
from openai import OpenAI
# from app.config import ROOT_DIR, DATA_DIR  # Ensure DATA_DIR is defined in your config

# Load environment variables from .env
load_dotenv(ROOT_DIR / '.env')

document_number = 46912

# Open the image file in binary mode and encode it as Base64.
with open(DATA_DIR / "images" / f"{document_number}.jpg", "rb") as f:
    image_bytes = f.read()

# Create a Base64 data URL from the image bytes.
encoded_image = base64.b64encode(image_bytes).decode("utf-8")
data_url = f"data:image/jpeg;base64,{encoded_image}"

# Initialize the OpenAI client with the API key.
client = OpenAI(
    api_key=os.getenv("OPENAI_TEST_KEY")
)

promt = """
Can you transcribe the text from the image? Also do your best effort to fix any issues with the document scanning procedure.
Please do your best to fill out all the information in the response object. If there are any issues or comments, please add them to "observations"

Please return the results in a json file with the following structure so it can be directly parsed by a script.
{
    "metadata": {
    "document_date": "...",
    "people_mentioned": ["person1", ....],
    "places_mentioned: ["place 1", ...],
    "document_title": "...",
    "document_description": "...
    "observations: "..."
    },
    "original_text": "...",
    "reviewed_text:
}


"""


# Create a response that includes both a text prompt and the image input.
response = client.responses.create(
    model="gpt-4o-mini",  # Change to "gpt-4o" if that's the vision-capable model available to you.
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": promt},
            {
                "type": "input_image",
                "image_url": data_url,
                "detail": "high"  # Use "low" for lower detail analysis if preferred.
            },
        ],
    }],
)

print(response.output_text)


ModuleNotFoundError: No module named 'dotenv'